### Курсовая работа по курсу "Рекомендательные системы"

- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [71]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [72]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=3000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 3001


In [73]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data['user_id'].nunique()} Items: {df_data['item_id'].nunique()}")

In [74]:
# общие пользователи
common_users = list(set(data_train_lvl_1.user_id.values)&(set(data_val_lvl_1.user_id.values))&set(data_val_lvl_2.user_id.values))

data_train_lvl_1 = data_train_lvl_1[data_train_lvl_1.user_id.isin(common_users)]
data_val_lvl_1 = data_val_lvl_1[data_val_lvl_1.user_id.isin(common_users)]
data_train_lvl_2 = data_train_lvl_2[data_train_lvl_2.user_id.isin(common_users)]
data_val_lvl_2 = data_val_lvl_2[data_val_lvl_2.user_id.isin(common_users)]

print_stats_data(data_train_lvl_1,'train_lvl_1')
print_stats_data(data_val_lvl_1,'val_lvl_1')
print_stats_data(data_train_lvl_2,'train_lvl_2')
print_stats_data(data_val_lvl_2,'val_lvl_2')

train_lvl_1
Shape: (784420, 13) Users: 1915 Items: 3001
val_lvl_1
Shape: (163261, 12) Users: 1915 Items: 27118
train_lvl_2
Shape: (163261, 12) Users: 1915 Items: 27118
val_lvl_2
Shape: (115989, 12) Users: 1915 Items: 24042


#### MainRecommender

In [75]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/3001 [00:00<?, ?it/s]

In [76]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,6,"[1024306, 1102949, 6548453, 835394, 940804, 96..."


In [77]:
def do_recommend(model, func, n, k):
    
    func_dic = {
        'get_als_recommendations': model.get_als_recommendations,
        'get_own_recommendations': model.get_own_recommendations,
        'get_similar_items_recommendation': model.get_similar_items_recommendation,
        'get_similar_users_recommendation': model.get_similar_users_recommendation
    }
    
    try:
        result = func_dic[func](n, k)
    
    except (IndexError, ValueError) as e:
        
        result = recommender._extend_with_top_popular([], k)
    
    return result

In [78]:
%%time 

k = 50
result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    recommender.get_own_recommendations(x, k))
result_lvl_1.head(2)

CPU times: user 6.88 s, sys: 141 ms, total: 7.02 s
Wall time: 7.07 s


,user_id,actual,own
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 9297615, 5577022, 9655212, 1124029, 8..."
1,6,"[1024306, 1102949, 6548453, 835394, 940804, 96...","[13003092, 972416, 1084036, 995598, 5569792, 1..."


In [79]:
result_lvl_1.apply(lambda row: precision_at_k(row['own'], row['actual'], k=5), axis=1).mean()

0.20553524804177548

### 2-lvl model

In [80]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2.head(2)

,user_id
0,2070
1,2021


In [81]:
# # только warm start
# train_users = data_train_lvl_1['user_id'].unique()
# users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=k))
users_lvl_2.head(4)

,user_id,candidates
0,2070,"[1105426, 944588, 1092937, 1008814, 917033, 83..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."
2,1753,"[963686, 1057168, 9553382, 942166, 935993, 106..."
3,2120,"[1029743, 1106523, 5569230, 916122, 1070820, 8..."


In [82]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)

users_lvl_2.head(4)

,user_id,item_id
0,2070,1105426
0,2070,944588
0,2070,1092937
0,2070,1008814


In [83]:
targets_lvl_2 = data_train_lvl_2.copy()
targets_lvl_2['target'] = 1  # тут только покупки 

In [84]:
targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

In [85]:
targets_lvl_2['target'].fillna(0, inplace= True)

In [86]:
targets_lvl_2['target'].mean()

0.1249599519423308

### XGBoost

In [87]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [88]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [89]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,944588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,LUNCHMEAT,HAM,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


#### FE

In [90]:
# добавим средний чек по каждому пользователю

basket_sum = targets_lvl_2.groupby(['user_id', 'basket_id'], as_index=False).agg({'sales_value': 'sum'})
basket_mean = basket_sum.groupby(['user_id'], as_index=False).agg({'sales_value': 'mean'}).\
            rename(columns={'sales_value':'basket_mean'})

targets_lvl_2 = targets_lvl_2.merge(basket_mean, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,15.38
1,2070,944588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,HAM,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,15.38


In [91]:
# кол-во покупок в каждой категрии по пользователям

user_cat_sale_count = targets_lvl_2.groupby(['user_id', 'commodity_desc'], as_index=True).\
                        agg({'commodity_desc': 'count'}).\
                        rename(columns={'commodity_desc':'commodity_count'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(user_cat_sale_count, on=['user_id', 'commodity_desc'], how='left')
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,15.38,2
1,2070,944588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,15.38,4


In [92]:
# кол-во покупок категории в неделю

commodity_week_count = targets_lvl_2.groupby(['commodity_desc', 'week_no'], as_index=True).\
                        agg({'commodity_desc': 'count'}).\
                        rename(columns={'commodity_desc':'commodity_week_count'}).\
                        groupby(['commodity_desc']).agg({'commodity_week_count': 'mean'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(commodity_week_count, on='commodity_desc', how='left')
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,15.38,2,2.666667
1,2070,944588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,15.38,4,44.333333


In [93]:
# стоимость товара к средней стоимости по категории

commodity_mean_price = targets_lvl_2.groupby(['commodity_desc', 'item_id'], as_index=True).\
                        agg({'sales_value': 'mean'}).\
                        rename(columns={'sales_value':'item_price'}).\
                        groupby(['commodity_desc']).agg({'item_price': 'mean'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(commodity_mean_price, on='commodity_desc', how='left')
targets_lvl_2['price_to_cat'] = targets_lvl_2[['sales_value', 'item_price']].\
                    apply(lambda row: row['sales_value']/row['item_price'], axis=1)
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count,item_price,price_to_cat
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50-74K,Unknown,Unknown,1,None/Unknown,15.38,2,2.666667,6.257500,NaN
1,2070,944588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50-74K,Unknown,Unknown,1,None/Unknown,15.38,4,44.333333,3.477929,NaN


In [151]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [122]:
X_train.columns

Index(['user_id', 'item_id', 'basket_id', 'day', 'quantity', 'sales_value',
       'store_id', 'retail_disc', 'trans_time', 'week_no', 'coupon_disc',
       'coupon_match_disc', 'manufacturer', 'department', 'brand',
       'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product',
       'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
       'hh_comp_desc', 'household_size_desc', 'kid_category_desc',
       'basket_mean', 'commodity_count', 'commodity_week_count', 'item_price',
       'price_to_cat'],
      dtype='object')

In [95]:
item_features.columns, user_features.columns

(Index(['item_id', 'manufacturer', 'department', 'brand', 'commodity_desc',
        'sub_commodity_desc', 'curr_size_of_product'],
       dtype='object'),
 Index(['age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
        'hh_comp_desc', 'household_size_desc', 'kid_category_desc', 'user_id'],
       dtype='object'))

In [152]:
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc',
       'sub_commodity_desc', 'curr_size_of_product', 'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
        'hh_comp_desc', 'household_size_desc', 'kid_category_desc']

other_feats = ['user_id', 'item_id', 'day', 'quantity', 'sales_value', 'retail_disc', 'trans_time', 
               'week_no', 'coupon_disc', 'coupon_match_disc',
       'basket_mean', 'commodity_count', 'commodity_week_count', 'item_price',
       'price_to_cat']

X_train = X_train[other_feats + cat_feats]
X_train = X_train.fillna('na')
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [97]:
# X_train = pd.get_dummies(X_train, columns=cat_feats, drop_first=True)

In [98]:
# после OHE попадаются колонки с символами [ ] < их не принимает xgb

# regex = re.compile(r"\[|\]|<", re.IGNORECASE)
# X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) 
#                               else col for col in X_train.columns.values]


In [99]:
# xgb_model = XGBClassifier(random_state=42,
#                          n_estimators=100,
#                           learning_rate=0.01
#                          )

In [100]:
# xgb_model.fit(X_train, y_train)

In [153]:
cat_model = CatBoostClassifier(random_seed=42, max_depth=3, n_estimators=500,
                              verbose=0, cat_features=cat_feats)

cat_model.fit(X_train, y_train)

In [154]:
# train_preds = xgb_model.predict_proba(X_train)

train_preds = cat_model.predict_proba(X_train)

In [155]:

[cat_model.feature_names_] + [cat_model.feature_importances_]

[['user_id',
  'item_id',
  'day',
  'quantity',
  'sales_value',
  'retail_disc',
  'trans_time',
  'week_no',
  'coupon_disc',
  'coupon_match_disc',
  'basket_mean',
  'commodity_count',
  'commodity_week_count',
  'item_price',
  'price_to_cat',
  'manufacturer',
  'department',
  'brand',
  'commodity_desc',
  'sub_commodity_desc',
  'curr_size_of_product',
  'age_desc',
  'marital_status_code',
  'income_desc',
  'homeowner_desc',
  'hh_comp_desc',
  'household_size_desc',
  'kid_category_desc'],
 array([2.85113201e-01, 0.00000000e+00, 6.85095762e+00, 6.40327471e+01,
        1.01801138e-02, 3.47726914e+00, 1.72093763e+01, 3.38846091e-03,
        4.70102686e+00, 2.63528010e+00, 1.40957850e-02, 0.00000000e+00,
        6.26157183e-01, 6.41454131e-02, 9.02627941e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00

In [156]:
targets_lvl_2['own_proba'] = train_preds[:,1]
targets_lvl_2.head(3)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count,item_price,price_to_cat,own_proba
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,Unknown,1,None/Unknown,15.38,2,2.666667,6.257500,NaN,0.000009
1,2070,944588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unknown,Unknown,1,None/Unknown,15.38,4,44.333333,3.477929,NaN,0.000009
2,2070,1092937,4.094147e+10,619.0,1.0,3.29,311.0,0.0,2015.0,89.0,...,Unknown,Unknown,1,None/Unknown,15.38,4,44.333333,3.477929,0.945965,0.999960


In [157]:
train_users_lvl2 = targets_lvl_2['user_id'].unique()
train_users_lvl2

array([2070, 2021, 1753, ...,  436, 1697, 1745])

In [158]:
def get_xgb_rec(uid, df):
    return df.loc[df['user_id']==uid][['item_id', 'own_proba']].\
                sort_values('own_proba', ascending=False).head(5).item_id.to_list()

In [159]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']
result_lvl_2.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [160]:
train_users_2 = users_lvl_2['user_id'].unique()
result_lvl_2 = result_lvl_2[result_lvl_2['user_id'].isin(train_users_2)]


In [161]:
result_lvl_2['candidates'] = result_lvl_2['user_id'].apply(lambda x: 
                                                          recommender.get_own_recommendations(x, N=5))
result_lvl_2.head(3)

,user_id,actual,candidates
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 9297615, 5577022, 9655212, 1124029]"
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13003092, 972416, 1084036, 995598, 5569792]"
2,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9338009, 939681, 974548, 894360, 1136033]"


In [162]:
result_lvl_2['xgb_candidates'] = result_lvl_2['user_id'].apply(lambda x: 
                                                          get_xgb_rec(x, targets_lvl_2))
result_lvl_2.head(3)

,user_id,actual,candidates,xgb_candidates
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 9297615, 5577022, 9655212, 1124029]","[940947, 9655212, 940947, 1082269, 979792]"
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13003092, 972416, 1084036, 995598, 5569792]","[6548453, 6548453, 1024306, 1024306, 1024306]"
2,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9338009, 939681, 974548, 894360, 1136033]","[9338009, 12524016, 1122358, 6554400, 855350]"


In [163]:
prec_1lvl_model = result_lvl_2['user_id'].apply(lambda x: 
                                    precision_at_k(result_lvl_2.loc[result_lvl_2['user_id']==x, 
                                                                          'candidates'].squeeze(), 
                                                   result_lvl_2.loc[result_lvl_2['user_id']==x, 
                                                                          'actual'].squeeze(), 
                                                   k=5)).mean()

prec_2lvl_model = result_lvl_2['user_id'].apply(lambda x: 
                                    precision_at_k(result_lvl_2.loc[result_lvl_2['user_id']==x, 
                                                                          'xgb_candidates'].squeeze(), 
                                                   result_lvl_2.loc[result_lvl_2['user_id']==x, 
                                                                          'actual'].squeeze(), 
                                                   k=5)).mean()

print(f'1-lvl model Precision is {prec_1lvl_model}\n2-lvl model Precision is {prec_2lvl_model}')

1-lvl model Precision is 0.1639686684073107
2-lvl model Precision is 0.1809921671018277


#### Test

In [176]:
data_test = pd.read_csv('data/retail_test1.csv')
data_test = prefilter_items(data=data_test, take_n_popular=18000, item_features=item_features)
data_test.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0,8.49
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0,6.29
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0,7.99
5,1602,41665647035,664,937330,1,3.29,334,-1.2,1741,96,0.0,0.0,3.29
7,1602,41665647035,664,1044259,1,3.99,334,0.0,1741,96,0.0,0.0,3.99


In [177]:
data_test = data_test[data_test['user_id'].isin(result_lvl_2['user_id'].unique())]

data_test

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,1340,41652823310,664,912987,1,8.49,446,0.00,52,96,0.0,0.0,8.49
1,588,41652838477,664,1024426,1,6.29,388,0.00,8,96,0.0,0.0,6.29
3,1602,41665647035,664,827939,1,7.99,334,0.00,1741,96,0.0,0.0,7.99
5,1602,41665647035,664,937330,1,3.29,334,-1.20,1741,96,0.0,0.0,3.29
7,1602,41665647035,664,1044259,1,3.99,334,0.00,1741,96,0.0,0.0,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88716,1705,41944916893,684,6554202,1,3.00,421,-0.69,1341,98,0.0,0.0,3.00
88717,1705,41944916893,684,6632622,1,4.99,421,0.00,1341,98,0.0,0.0,4.99
88718,1705,41944916893,684,6632695,1,4.99,421,0.00,1341,98,0.0,0.0,4.99
88724,98,41944918649,684,847241,1,2.99,421,0.00,1702,98,0.0,0.0,2.99


In [178]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 940947, 952924, 959219, 99102..."
1,6,"[847738, 908813, 948650, 992393, 1055911, 1082..."


In [179]:
%%time 

k = 50
result_test['als'] = result_test['user_id'].apply(lambda x: 
                                    recommender.get_own_recommendations(x, k))

CPU times: user 5.48 s, sys: 62.3 ms, total: 5.54 s
Wall time: 5.68 s


In [180]:
result_test['xgb_candidates'] = result_test['user_id'].apply(lambda x: 
                                                          get_xgb_rec(x, targets_lvl_2))
result_test.head(3)

,user_id,actual,als,xgb_candidates
0,1,"[880007, 883616, 940947, 952924, 959219, 99102...","[856942, 9297615, 5577022, 9655212, 1124029, 8...","[940947, 9655212, 940947, 1082269, 979792]"
1,6,"[847738, 908813, 948650, 992393, 1055911, 1082...","[13003092, 972416, 1084036, 995598, 5569792, 1...","[6548453, 6548453, 1024306, 1024306, 1024306]"
2,7,"[859987, 863407, 895454, 906636, 930918, 95467...","[9338009, 939681, 974548, 894360, 1136033, 557...","[9338009, 12524016, 1122358, 6554400, 855350]"


In [181]:
precision_k = result_test['user_id'].apply(lambda x: 
                                    precision_at_k(result_test.loc[result_test['user_id']==x, 
                                                                          'als'].squeeze(), 
                                                   result_test.loc[result_test['user_id']==x, 
                                                                          'actual'].squeeze(), 
                                                   k=5)).mean()

precision_k_xgb = result_test['user_id'].apply(lambda x: 
                                    precision_at_k(result_test.loc[result_test['user_id']==x, 
                                                                          'xgb_candidates'].squeeze(), 
                                                   result_test.loc[result_test['user_id']==x, 
                                                                          'actual'].squeeze(), 
                                                   k=5)).mean()

precision_k, precision_k_xgb

(0.13957816377171217, 0.14193548387096774)